In [4]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.tests.datafiles import TPR, XTC

In [5]:
u = mda.Universe(TPR, XTC, in_memory=True)
protein = u.select_atoms("protein")

In [6]:
prealigner = align.AlignTraj(u, u, select="protein and name CA",
                             in_memory=True).run()

In [7]:
ref_coordinates = u.trajectory.timeseries(asel=protein).mean(axis=1)

In [8]:
ref = mda.Merge(protein).load_new(ref_coordinates[:, None, :],
                                  order="afc")

In [9]:
aligner = align.AlignTraj(u, ref, select="protein and name CA",
                          in_memory=True).run()
# need to write the trajectory to disk for PMDA 0.3.0 (see issue #15)
with mda.Writer("rmsfit.xtc", n_atoms=u.atoms.n_atoms) as W:
    for ts in u.trajectory:
        W.write(u.atoms)

In [13]:
from MDAnalysis.analysis.rms import RMSF

u = mda.Universe(TPR, "rmsfit.xtc")
calphas = protein.select_atoms("protein and name CA")

rmsfer = RMSF(calphas).run()